# Add Dependences

In [226]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [227]:
parameters = Model(
    Cubic(
        μ₁       = 0.003,
        rₘᵢₙ     = 2,
        rₘₐₓ     = 3
    ),
    Contractile(
        fₚ       = 0.001
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 50,
        nn       = 12
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0;
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "Test-fusion.xyz"
    )
)

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)


 Loading Force Function
  0.000016 seconds (17 CPU allocations: 1.237 KiB)


force (generic function with 1 method)

## Starting Data

In [228]:
# Loading Data
CUDA.@time X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
# Generating Struct
CUDA.@time Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

  0.000799 seconds (8.06 k CPU allocations: 317.672 KiB) (1 GPU allocation: 6.305 KiB, 2.32% memmgmt time)
  0.131634 seconds (346.33 k CPU allocations: 20.766 MiB) (12 GPU allocations: 4.813 MiB, 0.06% memmgmt time)


## run_event -> one_aggregate

In [229]:
# Simulation One Aggregate
# CUDA.@time one_aggregate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

## run_event -> fusion

In [230]:
# Simulating Two Aggregate
CUDA.@time fusion(true, parameters, Cubic(100, 2.0, 3.0), Agg)

Finding Equilibrium in one Aggregate


Progress: 100%|█████████████████████████| Time: 0:03:21


Finding Equilibrium in two Aggregate


Progress: 100%|█████████████████████████| Time: 0:04:37


479.508772 seconds (1.22 G CPU allocations: 66.986 GiB, 4.18% gc time) (25.40 M GPU allocations: 5.605 TiB, 18.38% memmgmt time)
